<a href="https://colab.research.google.com/github/linlih/CS-Notes/blob/master/code/BackPropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 反向传播代码示例

参考自：https://www.cnblogs.com/charlotte77/p/5629865.html

In [0]:
import random
import math

In [0]:
"""
参数说明：
"pd_":偏导的前缀，partial derivative
"d_" :倒数的前缀，derivative
"w_ho": 隐含层到输出层的权重系数索引
"w_ih": 输出层到隐含层的权重系数索引
"""

In [0]:
class NeuralNetwork:
  LEARING_RATE = 0.5
  
  def __init__(self, num_inputs, num_hidden, num_outputs, hidden_layer_weights = None, hidden_layer_bias = None, output_layer_weights = None, output_layer_bias = None):
    self.num_inputs = num_inputs
    self.hidden_layer = NeuronLayer(num_hidden, hidden_layer_bias)
    self.output_layer = NeuronLayer(num_outputs, output_layer_bias)
    self.init_weights_from_inputs_to_hidden_layer_neurons(hidden_layer_weights)
    self.init_weights_from_hidden_layer_neurons_to_output_layer_neurons(output_layer_weights)
  
  def init_weights_from_inputs_to_hidden_layer_neurons(self, hidden_layer_weights):
    weight_num = 0
    for h in range(len(self.hidden_layer.neurons)):
      for i in range(self.num_inputs):
        if not hidden_layer_weights:
          self.hidden_layer.neurons[h].weights.append(random.random())
        else:
          self.hidden_layer.neurons[h].weights.append(hidden_layer_weights[weight_num])
          weight_num += 1
        
  def init_weights_from_hidden_layer_neurons_to_output_layer_neurons(self, output_layer_weights):
    weight_num = 0
    for o in range(len(self.output_layer.neurons)):
      for h in range(len(self.hidden_layer.neurons)):
        if not output_layer_weights:
          self.output_layer.neurons[o].weights.append(random.random())
        else:
          self.output_layer.neurons[o].weights.append(output_layer_weights[weightnum])
          weight_num += 1
  
  def inspect(self):
    print('-----------------')
    print("Input:{}".format(self.num_input))
    print('-----------------')
    print("Hidden Layer")
    self.hidden_layer.inspect()
    print('-----------------')
    print("Output Layer")
    self.output_layer.inspect()
    print('-----------------')

  def feed_forward(self, inputs):
    hidden_layer_outputs = self.hidden_layer.feed_forward(inputs)
    return self.output_layer.feed_forward(hidden_layer_outputs)

  def train(self, training_inputs, training_outputs):
    self.feed_forward(training_inputs)
  
    # 1.输出神经元的值
    pd_errors_wrt_output_neuron_total_net_input = [0] * len(self.output_layer.neurons)
    for o in range(len(self.output_layer.neurons)):
      # ∂E/∂zⱼ
      pd_errors_wrt_output_neuron_total_net_input[o] = self.output_layer.neurons[o].calculate_pd_error_wrt_total_net_input(training_outputs[o])
    
    # 2.隐含层神经元的值
    pd_error_wrt_hidden_neuron_total_net_input = [0] * len(self.hidden_layer.neurons)
    for h in range(len(self.hidden_layer.neurons)):

      # dE/dyⱼ = Σ ∂E/∂zⱼ * ∂z/∂yⱼ = Σ ∂E/∂zⱼ * wᵢⱼ
      d_error_wrt_hidden_neuron_output = 0
      for o in range(len(self.output_layer.neurons)):
        d_error_wrt_hidden_neuron_output += pd_error_wrt_output_neuron_total_net_input[o] * self.output_layer.neurons[o].weights[h]
        # ∂E/∂zⱼ = dE/dyⱼ * ∂zⱼ/∂
        pd_errors_wrt_output_neuron_total_net_input[h] = d_error_wrt_hidden_neuron_output * self.hidden_layer.neurons[h].calculate_pd_total_net_input_wrt_input()

    # 3.更新输出层权重系数
    for o in range(len(self.output_layer.neurons)):
      for w_ho in range(len(self.output_layer.neurons[o].weights)):
        pd_error_wrt_weight = pd_errors_wrt_output_neuron_total_net_input[o] * self.output_layer.neurons[o].calculate_pd_total_net_input_wrt_weight(w_ho)

        self.output_layer.neurons[o].weights[w_ho] -= self.LEARING_RATE * pd_error_wrt_weight
    
    # 4.更新隐含层的权重系数
    for h in range(len(self.hidden_layer.neurons)):
      for w_ih in range(len(self.hidden_layer.neurons[h].weights)):
        pd_error_wrt_weight = pd_error_wrt_hidden_neuron_total_net_input[h] * self.hidden_layer.neurons[h].calculate_pd_total_net_input_wrt_weight(w_ih);
        self.hidden_layer.neurons[h].weights[w_ih] -= self.LEARING_RATE * pd_error_wrt_weight
  
  def calculate_total_error(self, training_sets):
    total_error = 0
    for t in range(len(training_sets)):
      training_inputs, training_outputs = training_sets[t]
      self.feed_forward(training_inputs)
      for o in range(len(training_outputs[o])):
        total_error += self.output_layer.neurons[o].calculate_error(training_outputs[o])
    return total_error


In [0]:
class 